In [1]:
import glob
import pandas as pd
import numpy as np
import os, sys
import collections
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier 

In [3]:
from numba.typed import List
from numba import jit, njit, vectorize

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [5]:
sample_rate = 50  # number of observation per second based on dataset documentation(150 samples in 3 second)

sliding_size = int((1/3) * sample_rate)  # number of skipped datapoints to start next window
print(sliding_size)

16


# Feature sets

In [6]:
@njit()
def mean_crossing_rate(col):
    # col = np.array(values)
    normalized = col - col.mean()  # to make elements of array possitive or negetive
    return ((normalized[:-1] * col[1:]) < 0).sum()  # Zero-Crossing_rate

@njit()
def iqr(window):  # inter-quartile range
    Q1 = np.median(window[:len(window)//2])  # First quartile (Q1) 
    Q3 = np.median(window[len(window)//2:])  # Third quartile (Q3) 
    IQR = Q3 - Q1 # Interquartile range (IQR) 
    return(IQR) 
@njit()
def calc_sma_for_window(data):
    return np.sum(data) / len(data)  
@njit()
def get_min(x):
    m = np.min(x)
    return m
@njit()
def get_max(x):
    m = np.max(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_var(x):
    m = np.var(x)
    return m
@njit()
def get_sum(x):
    m = x.sum()
    return m 
@njit()
def get_median(x):
    m = np.median(x)
    return m 
@njit()
def get_std(x):
    m = np.median(x)
    return m 
@njit()
def get_rng(x):
    n = np.max(x)
    m = np.min(x)
    z = n-m
    return z 

def get_entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

In [7]:
def Energy(frame):
    return sum( [ abs(x)**2 for x in frame ] ) / len(frame)

In [8]:
def Features(window):# mean, std,max,min and zero-crossing-rate
    wind = window.iloc[:, :-1]
    win = np.array(wind)
    features = []
    
    features.append(get_mean(win))
    features.append(get_median(win))
    features.append(get_std(win))
    features.append(get_min(win))
    features.append(get_max(win))
    features.append(get_sum(win))
    features.append(calc_sma_for_window(win))
    features.append(get_rng(win))
    features.append(get_entropy(win))
    mean_crossing = [mean_crossing_rate(win[:, i]) for i in range(win.shape[1])]
    features.append(np.array(mean_crossing))
    IQR = iqr(win)
    features.append(np.array(IQR))
    energy_measure = Energy(win)
    features.append(np.array(energy_measure))
 

    
    features = np.hstack(features).tolist()
    
    label = window.iloc[:, -1].mode()[0]  ## select the most frequent label as the label of the window
    features.append(label)
    return features

In [9]:
def windowing_dataset(dataset, win_size, feature_extraction_function, subject_id, overlap=False):
    windowed_dataset = []
    win_count = 0
    if overlap:
        step_size = sliding_size  # for Overlapping technique
    else:
        step_size = win_size  # for Non-overlapping technique

    for index in range(0, dataset.shape[0], step_size):
        start = index
        end = start + win_size
        # to assure all of windows are equal in size
        if (end <= dataset.shape[0]):
            window = dataset.iloc[start:end, :].reset_index(drop=True)
            win_count = win_count + 1
            features = feature_extraction_function(window)

            windowed_dataset.append(features)

    final = pd.DataFrame(windowed_dataset)
    final.insert(0, 'group', subject_id)  # to use in Subject CV
    return final

In [10]:
def Preprocessing(dataset_path, overlapping):
    feature_function = Features
    win_size = 3

    print("Start for win size {}".format(win_size))
    datapoints_per_window = int(win_size * sample_rate)

    print(feature_function.__name__)

    ALL = []
   
    for subject in range(1,18):
        file_path = dataset_path + '\subject{0}_ideal.csv'.format(subject)
        all_cols = []
        
        
#########################################################################################################################
        for i in range(2, 117, 13):# indices of accelarations
            indices = list(range(i, i + 13))
            all_cols.extend(indices)

        all_cols.append(119)  # label index

        tmp_db = pd.read_csv(file_path, header=None, usecols=all_cols, sep='\t')
        tmp_db.columns = list(range(tmp_db.shape[1]))  # re-index the columns

        transformed_db = windowing_dataset(tmp_db, datapoints_per_window, feature_function, subject,
                                                   overlap=overlapping)

        ALL.append(transformed_db)
#########################################################################################################################
       
    #final_dataset = pd.DataFrame()
    ALL_dataset = pd.DataFrame()
    ALL_dataset = ALL_dataset.append(ALL, ignore_index=True)
    
    return ALL_dataset
   

In [11]:
def subject_cross_validation(X, Y, groups, classifier):
    f1 = []
    logo = LeaveOneGroupOut()
    i = 0
    for train_index, test_index in logo.split(X, Y, groups=groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        i += 1
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        f = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
        
        print('Model-',i ,' -',' f1 score: ', f)
       
        f1.append(f)
    return np.mean(f1)

# mean_removal on features

In [12]:
def classifier(dataset, model):
    results = dict()
    win_size = float(3)
    print('window_size = ', win_size,' sec')

    dataset = dataset
    groups = dataset['group']
    X = dataset.iloc[:, 1:-1]
    X = np.array(X)
    data_standardized = preprocessing.scale(X) # mean removal on features after extraction
    X = data_standardized
    Y = dataset.iloc[:, -1]
    Y = np.array(Y)

    for model_name, mod in model.items():
        f1 = 0

        f1 = subject_cross_validation(X, Y, groups, mod)

        if win_size in results:
            results[win_size].append(f1)
        else:
            results[win_size] = [f1]


        results = collections.OrderedDict(sorted(results.items()))

        final = []
        col = list(model.keys())
        col.insert(0, "window-size")
        final.append(col)
        for k, v in results.items():
            tmp = []
            tmp.append([k])
            tmp.append(v)
            flattened = [val for sublist in tmp for val in sublist]
            final.append(flattened)

    accuracy = final[1][1]
    print(accuracy)
    return accuracy

In [22]:
mod = {'RF': RandomForestClassifier(n_estimators=40, random_state=42, n_jobs=-1)}

In [14]:
dataset_csv_path = r"D:\projec\proj\data"

overlapping = 1  # input 0 for non overlapping, 1 for overlapping

ALL = Preprocessing(dataset_path=dataset_csv_path, overlapping=bool(int(overlapping)))

Start for win size 3
Features


In [15]:
ALL.head(10)

,group,0,1,2,3,4,5,6,7,8,...,235,236,237,238,239,240,241,242,243,244
0,1,-0.317008,0.074480,0.074480,-13.572,13.123,-5563.499014,-37.089993,26.695,7.228819,...,3.433057,0.096471,0.409673,0.065249,0.081963,0.060492,0.428163,0.104005,0.407340,1
1,1,-0.320368,0.068405,0.068405,-13.572,13.123,-5622.463930,-37.483093,26.695,7.228819,...,2.955452,0.095075,0.413741,0.068959,0.064359,0.058963,0.406512,0.092348,0.442178,1
2,1,-0.323965,0.068771,0.068771,-13.572,13.123,-5685.586551,-37.903910,26.695,7.228819,...,2.090189,0.071682,0.437328,0.062415,0.059816,0.050159,0.424356,0.094437,0.431047,1
3,1,-0.336631,0.071631,0.071631,-13.547,10.304,-5907.879152,-39.385861,23.851,7.228819,...,2.260539,0.066683,0.459112,0.052358,0.067397,0.045673,0.462147,0.108078,0.384102,1
4,1,-0.348977,0.073038,0.073038,-13.547,12.127,-6124.541712,-40.830278,25.674,7.228819,...,3.138315,0.075594,0.467472,0.047577,0.067944,0.051054,0.470697,0.113735,0.364515,1
5,1,-0.371705,0.086086,0.086086,-16.693,13.038,-6523.425930,-43.489506,29.731,7.228819,...,3.324015,0.098932,0.452130,0.049836,0.075622,0.054213,0.458554,0.109609,0.377625,1
6,1,-0.397238,0.088222,0.088222,-16.693,13.038,-6971.534233,-46.476895,29.731,7.228819,...,3.237408,0.099800,0.440110,0.051009,0.077333,0.055490,0.446425,0.105752,0.392333,1
7,1,-0.435878,0.082128,0.082128,-17.479,18.240,-7649.662100,-50.997747,35.719,7.228819,...,3.112343,0.092486,0.455538,0.052813,0.058612,0.055669,0.429769,0.098172,0.416389,1
8,1,-0.467565,0.093603,0.093603,-18.348,18.240,-8205.770461,-54.705136,36.588,7.228819,...,2.196212,0.069423,0.463414,0.048845,0.073784,0.048486,0.451738,0.104297,0.395479,1
9,1,-0.503556,0.096744,0.096744,-18.348,18.240,-8837.401092,-58.916007,36.588,7.228819,...,3.605017,0.084708,0.479525,0.042276,0.069691,0.051493,0.466764,0.110999,0.370745,1


In [23]:
print('ALL_accuracy = ', )
ALL_accuracy = classifier(dataset=ALL, model=mod)
ALL_accuracy 

ALL_accuracy = 
window_size =  3.0  sec


C:\Users\BKY\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:191: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


Model- 1  -  f1 score:  0.9748251748251748
Model- 2  -  f1 score:  0.984843144166373
Model- 3  -  f1 score:  0.9628
Model- 4  -  f1 score:  0.9807251032583754
Model- 5  -  f1 score:  0.9679310344827586
Model- 6  -  f1 score:  0.96
Model- 7  -  f1 score:  0.9944598337950139
Model- 8  -  f1 score:  0.956221198156682
Model- 9  -  f1 score:  0.9620512820512821
Model- 10  -  f1 score:  0.9771252987367702
Model- 11  -  f1 score:  0.9665625
Model- 12  -  f1 score:  0.9729899497487438
Model- 13  -  f1 score:  0.9652836579170194
Model- 14  -  f1 score:  0.9489616332277367
Model- 15  -  f1 score:  0.9718492854049372
Model- 16  -  f1 score:  0.922936449963477
Model- 17  -  f1 score:  0.9566375047546596
0.9662472382640591


0.9662472382640591

In [24]:
d ={'ALL': [ALL_accuracy]}

In [25]:
df_accuracy = pd.DataFrame(data=d)

In [26]:
df_accuracy 

,ALL
0,0.966247
